In [13]:
# !pip install langchain langchain_community
# !pip install fitz
# !pip install pymupdf
# !pip install faiss-cpu
# !pip install -qU langchain-huggingface
# !pip install langchain_groq
# ! pip install langchain_ollama

In [5]:
import os
from tqdm import tqdm
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyMuPDFLoader
# from langchain_ollama import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import time

## load the Groq API key
groq_api_key=os.environ['GROQ_API_KEY'] = "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"


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# embeddings = OllamaEmbeddings(model="llama3")
embeddings = HuggingFaceEmbeddings(model_name="/content/drive/MyDrive/embeddings/cache/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/8b3219a92973c328a8e22fadcfa821b5dc75636a")
text_splitter=RecursiveCharacterTextSplitter(chunk_size=700,chunk_overlap=200)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
def pdf_page_reader(book, start_page, end_page):
  '''
  This Function will extract the text from the given start page and end page of the pdf
  '''
  try:
    loader = PyMuPDFLoader(book)
    data = loader.load()
    number_of_pages = len(data)
  except:
    print("some problem with the pdf, please check")
  if start_page >= 0 and end_page < number_of_pages:
    required_data = []
    for i in tqdm(range(start_page, end_page+1)):
      doc = data[i]
      required_data.append(doc)
    return required_data
  else:
    return "Invalid Range"

pdf_path = "/content/drive/MyDrive/Concepts of Biology.pdf"
pdf_data = pdf_page_reader(pdf_path, 89, 132)

100%|██████████| 44/44 [00:00<00:00, 57047.72it/s]


In [9]:
final_documents=text_splitter.split_documents(pdf_data)
vectors=FAISS.from_documents(final_documents,embeddings)

In [12]:
vectors.save_local("/content/drive/MyDrive/faiss_db")

In [16]:
llm=ChatGroq(groq_api_key=groq_api_key,
model_name="mixtral-8x7b-32768")
prompt=ChatPromptTemplate.from_template(
"""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}
"""
)
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [21]:
text = "what is plasma membrane"
response=retrieval_chain.invoke({"input":text})


In [22]:
response["answer"]

"The plasma membrane, also known as the cell membrane, is the outermost layer of a cell that separates the cell's interior from its external environment. It is composed of a bilayer of phospholipids, with their hydrophobic, fatty acid tails in contact with each other. The landscape of the membrane is studded with proteins, some of which span the membrane and serve to transport materials into or out of the cell. Carbohydrates are attached to lipids or proteins on the exterior surface of the plasma membrane, forming complexes that help the cell bind substances it needs from the extracellular fluid. The plasma membrane has both hydrophilic and hydrophobic regions, which allows the movement of certain materials through the membrane while hindering the movement of others. It is not a static structure, but rather a fluid mosaic that is able to move and change in response to the cell's needs."